In [93]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
class BinaryClassifer:
    
    
    def __init__(self,learning_rate=0.01,iterations=1000):
        
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None
        
    def sigmoid(self,z):
        return (1/(1+np.exp(-z)))
        
    def fit(self,X,y):
        
        samples,features = X.shape
        self.weights = np.zeros(features)
        self.bias = 0 
        
        for _ in range(self.iterations):
            
            linear_model  = X @ self.weights + self.bias
            
            y_pred = self.sigmoid(linear_model)
            
            dw = (1/samples) * np.dot(X.T,(y_pred-y))
            db = (1/samples) * np.sum(y_pred-y)
            
            
            self.weights = self.weights - self.learning_rate * dw
            self.bias = self.bias - self.learning_rate * db
            

    def predict(self,X):
        return np.where(self.sigmoid(X @ self.weights + self.bias)>=0.5,1,0)
    
    
class Multiclass_Classifier:
    
    
    def __init__(self,learning_rate=0.01,iterations=10000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.classes = None
        self.classifiers = {}
        
    def fit(self,X,y):
        
        total_classes = np.unique(y)
        self.classes = total_classes
        
        for clss in total_classes:
            
            y_binary = np.where(y==clss,1,0)
            clf = BinaryClassifer(self.learning_rate,self.iterations)
            clf.fit(X,y_binary)
            self.classifiers[clss] = clf
            
    def predict(self,X):
        
        total_classes = self.classifiers.keys()
        predictions = []
        
        for cl in total_classes:
            proba = self.classifiers[cl].predict(X)
            predictions.append(proba)
            
        predictions = np.array(predictions)
        predictions = predictions.T
        
        return self.classes[np.argmax(predictions,axis=1)]

class Softmax_Classifier:
    def __init__(self,iterations=10000,learning_rate=0.01):
        
        self.iterations = iterations 
        self.learning_rate = learning_rate
        self.weights = None
        self.bias = None
        self.classes = None 
        
    def softmax_(self,z):
        
        z-= np.max(z,axis=1,keepdims=True)
        exp_z = np.exp(z)
        return exp_z/np.sum(exp_z,axis=1,keepdims=True)
        
    def one_hot_encoding(self,y,num_classes):
        
        one_hot_matrix = np.zeros((len(y),num_classes))
        one_hot_matrix[np.arange(len(y)),y]=1
        return one_hot_matrix
        
        
    def fit(self,X,y):
        
        samples,features = X.shape
        
        self.classes = np.unique(y)
        
        n_classes = len(self.classes)
        
        classes_to_index = {cls:index for index,cls in enumerate(self.classes)}
        
        y_index = np.array([classes_to_index[i] for i in y])
        
        y_one_hot = self.one_hot_encoding(y_index,n_classes)
        
        self.weights = np.zeros((features,n_classes))
        self.bias = np.zeros((1,n_classes))
        
        for _ in range(self.iterations):
            
            Z = X @ self.weights + self.bias
            y_pred = self.softmax_(Z)
            
            
            dw = (1/samples) * (X.T) @ (y_pred - y_one_hot)
            db = (1/samples) * np.sum(y_pred-y_one_hot)
            
            self.weights-= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
    def predict(self,X):
        pred = X @ self.weights + self.bias 
        proba = self.softmax_(pred)
        predictions = np.argmax(proba,axis=1)
        return self.classes[predictions]        
            
            
        
        
        

### Binary Classifier

In [84]:
X,y = load_breast_cancer(return_X_y=True)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [85]:
model = BinaryClassifer()

model.fit(X_train,y_train)

/tmp/ipykernel_5772/3265970859.py:12: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-z)))


In [86]:
y_pred = model.predict(X_test)

/tmp/ipykernel_5772/3265970859.py:12: RuntimeWarning: overflow encountered in exp
  return (1/(1+np.exp(-z)))


In [87]:
y_pred.shape

(114,)

In [88]:
y_test.shape

(114,)

In [89]:
mean_squared_error(y_pred,y_test)

0.05263157894736842

### MultiNomial Logistic Regression OvR(One-vs-Rest) Method

In [90]:
digits = load_digits()

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [91]:
model = Multiclass_Classifier()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

mean_squared_error(y_pred,y_test)

2.5201668984700976

### Using Softmax 

In [92]:
model = Softmax_Classifier()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

mean_squared_error(y_pred,y_test)

0.5618915159944368